<a href="https://colab.research.google.com/github/yvrjsharma/JAX/blob/main/JAX_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JAX: PMaps

JAX has lots of cool features to evaluate your code parallely. This is also called SPMD, or Single-Program Multiple-Data code. In this technique same code or computation is run in parallel on different input data on different devices (e.g. TPUs)

You can use pmap() to write a piece of code suitable for running on one device as well as on multiple devices.



Lets start using TPUs for this one. If you are using this Colab notebook, make sure you change your Runtime to TPU.

In [2]:
#Lets import the required libraries
import jax
import jax.numpy as jnp
import numpy as np

#transformation : higher order functions which take fuun as an input and outputs a transformed fun 
from jax import grad, jit, vmap, pmap
from jax import random

import matplotlib.pyplot as plt
from copy import deepcopy
from typing import Tuple, NamedTuple
import functools

In [3]:
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

In [4]:
import jax
jax.devices() #Eight devices

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In order to ustilize all the cores that are available to us, lets create some array with batch size equal to the number of cores available.

Lets now perform a dummy computation - convolving a small array over this array which is spread across the cores. This dummy example will help us in understanding later how pmap() helps in speeding up more complex or bigger computations.  

In [13]:
n_devices = jax.local_device_count()
x = np.arange(5)
xs = np.arange(5*n_devices).reshape(-1,5)  #40 unique values, arranged in 5 columns and 8 rows
w = np.array([2.,3.,4.])  #for convolving over the given metric
ws = np.stack([w] * n_devices)  #duplicating the same convolution kernel on 8 cores
x,w , xs, ws

(array([0, 1, 2, 3, 4]), array([2., 3., 4.]), array([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39]]), array([[2., 3., 4.],
        [2., 3., 4.],
        [2., 3., 4.],
        [2., 3., 4.],
        [2., 3., 4.],
        [2., 3., 4.],
        [2., 3., 4.],
        [2., 3., 4.]]))

In [12]:
#Simple convolution example 
def convolve(w,x):
  output = []
  for i in range(1,len(x)-1):
    output.append(jnp.dot(x[i-1:i+2],w))
  return jnp.array(output)

convolve(w,x) #smaller array is convolved over another one

DeviceArray([11., 20., 29.], dtype=float32)

In [15]:
#Lets first try the convolve operation over xs using just one device
#We can do so using vmap()
#convolve(ws,xs) #Error for incompatible shapes
jax.vmap(convolve)(ws,xs)

DeviceArray([[ 11.,  20.,  29.],
             [ 56.,  65.,  74.],
             [101., 110., 119.],
             [146., 155., 164.],
             [191., 200., 209.],
             [236., 245., 254.],
             [281., 290., 299.],
             [326., 335., 344.]], dtype=float32)

In [16]:
#Coolest thing in JAX is that you don't hav to change the function 
#When you need to run it on multiple devices
#Using pmap() this time
jax.pmap(convolve)(ws,xs)

ShardedDeviceArray([[ 11.,  20.,  29.],
                    [ 56.,  65.,  74.],
                    [101., 110., 119.],
                    [146., 155., 164.],
                    [191., 200., 209.],
                    [236., 245., 254.],
                    [281., 290., 299.],
                    [326., 335., 344.]], dtype=float32)

In [18]:
#Another way of writing above code
#Using in_axes argument of pmap() 
#Works similar to vmap()
jax.pmap(convolve,in_axes=(None,0))(w,xs)  #Using w instead of ws

#'None' would mean to broadcast weight w across all the cores 
#'0' would mean that 0 dimension in xs matrix is the batch argument
#So, pmap will replicate w and distribute every row in xs to a device

ShardedDeviceArray([[ 11.,  20.,  29.],
                    [ 56.,  65.,  74.],
                    [101., 110., 119.],
                    [146., 155., 164.],
                    [191., 200., 209.],
                    [236., 245., 254.],
                    [281., 290., 299.],
                    [326., 335., 344.]], dtype=float32)

* One thing to notice here is that now we have '**Sharded Device Arrays**' as an output instead if Device Array. This means that the data as well as the computation is sharded or distributed across multiple cores.

* Another thing to note here is that if we were to run another parallel computation, the elements would stay on their respective devices and that there won't be any cost incurred for moving data between devices.

* One more interesting thing to note here is that ```jax.pmap()``` does ```jit``` compile of the code additionally as part of its operations.

* In this example the array of five elements was distributed on every device, along with the single Kernel, there is **no communication happening across the multiple cores**.

We can choose while parallelizing the computation to have a communication between the multiple cores or not. We might have to train our Neural Netwrok model in a distributed fashion in which every core would receive a batch of data and then they will compute, communicate and cordinate among them to get the mean of gradients to update our ML model.

See below example for calculating the normalized values across each device so that the sum is 1. Here we need to pass the information across the devices.

In [20]:
#Using same convolution example 
#Comunicate across devices this time
def normalized_convolve(w,x):
  output = []
  for i in range(1,len(x)-1):
    output.append(jnp.dot(x[i-1:i+2],w))
  output = jnp.array(output)

  return output / jax.lax.psum(output,axis_name= 'p')  #Trying to communicate across devices

jax.pmap(normalized_convolve,axis_name='p', in_axes=(None,0))(w,xs) #If you will sum all the row values for any column you will get 1
#Note the Sharded Device Array as the output 

ShardedDeviceArray([[0.00816024, 0.01408451, 0.019437  ],
                    [0.04154303, 0.04577465, 0.04959785],
                    [0.07492582, 0.07746479, 0.07975871],
                    [0.10830861, 0.10915492, 0.10991956],
                    [0.14169139, 0.14084506, 0.14008042],
                    [0.17507419, 0.17253521, 0.17024128],
                    [0.20845698, 0.20422535, 0.20040214],
                    [0.24183977, 0.23591548, 0.23056298]], dtype=float32)

**value_and_grad()** function 
* It conveniently gives you both the function's value as well as its gradient.
* It will help in creating the model training pipeline using pure jax while getting the value as well as the gradient of the loss function. 

In [28]:
from jax._src.api import value_and_grad
#Simple loss function - sum of square of error differences 
def sum_squared_error(x,y):
  return sum((x-y)**2)  #derivative wrto x will be 2(x-y)

x = jnp.arange(4, dtype=jnp.float32)
y = x+0.4
print("x:",x,", y:",y, ", loss:", sum_squared_error(x,y))

#Calculating the loss value as well as gradient
jax.value_and_grad(sum_squared_error)(x,y)

x: [0. 1. 2. 3.] , y: [0.4 1.4 2.4 3.4] , loss: 0.6400001347064972


(DeviceArray(0.64000016, dtype=float32),
 DeviceArray([-0.8       , -0.79999995, -0.8000002 , -0.8000002 ], dtype=float32))

#### **Working towards training a Neural Network**

Training a model completly in parallel. 
If you want to log both, your loss value and your gradients of the loss then you can us jax.value_and_grad() inbuilt function.

All functions in jax are implemented as composale program/function transformations.

vmap automatically batches your code.
